In [1]:
import OpenBlender
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [3]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

In [4]:
from functions import textTransform
import yfinance as yf

In [5]:
os.chdir("..")

In [6]:
os.getcwd()

'/Users/yingyinxiao/Documents/Github/Stock-Market-Prediction/MSFT'

In [7]:
df = pd.read_csv("data/df_v2.csv").drop(columns = ["Unnamed: 0", "Unnamed: 0.1"], axis = 1)
df.head()

,date,volume,price,high,low,open,change,timestamp,FOXBUSINES.text_COUNT_last1days:microsoft,FOXBUSINES.text_last1days:microsoft,transformedDoc,FOXBUSINES.text_COUNT_last1days:negative,FOXBUSINES.text_last1days:negative,WALL_STREE.text_last1days:positive,WALL_STREE.text_COUNT_last1days:positive
0,02-01-2013 16:00:00,52.91M,27.62,27.73,27.15,27.25,3.41%,1.357142e+09,0,[],NaN,0,[],['the latest tweets and updates about the #fis...,6
1,03-01-2013 16:00:00,48.30M,27.25,27.65,27.16,27.63,-1.34%,1.357229e+09,2,['stocks ease after yesterdays big rally econo...,stocks ease after yesterdays big rally economy...,4,['amazons third-party sellers scored big this ...,['breaking: new jersey gov_ christie says hous...,3
2,04-01-2013 16:00:00,52.52M,26.74,27.34,26.73,27.27,-1.87%,1.357315e+09,2,['stocks mildly lower as traders mull mostly i...,stocks mildly lower as traders mull mostly inl...,2,['stocks mildly lower as traders mull mostly i...,['mark zuckerberg gave 498_8 million in facebo...,10
3,07-01-2013 16:00:00,37.12M,26.69,26.88,26.64,26.77,-0.19%,1.357574e+09,1,['stock futures edge lower after last weeks su...,stock futures edge lower after last weeks surg...,1,['stock futures edge lower after last weeks su...,"['buy a dvd or book, youve got something to pa...",5
4,08-01-2013 16:00:00,44.70M,26.55,26.79,26.46,26.75,-0.52%,1.357661e+09,0,[],NaN,0,[],"['@danahaggard ""assume positive intent"" seems ...",4


In [8]:
df.dtypes 
# change the type of "change" to float
# change the type of "date" to timestamp
# change the type of "volume" to float

date                                          object
volume                                        object
price                                        float64
high                                         float64
low                                          float64
open                                         float64
change                                        object
timestamp                                    float64
FOXBUSINES.text_COUNT_last1days:microsoft      int64
FOXBUSINES.text_last1days:microsoft           object
transformedDoc                                object
FOXBUSINES.text_COUNT_last1days:negative       int64
FOXBUSINES.text_last1days:negative            object
WALL_STREE.text_last1days:positive            object
WALL_STREE.text_COUNT_last1days:positive       int64
dtype: object

## Feature Engineering

In [9]:
df['change'] = df['change'].apply(lambda x: eval(x[:-1]))
df['date'] = pd.to_datetime(df['date'])

In [10]:
for i in df.volume:
    if i[-1] != 'M':
        print(i[-1])

K


In [11]:
def changeVolumeType(v):
    if v[-1] == 'M':
        return eval(v[:-1])
    if v[-1] == 'K':
        return eval(v[:-1])*10**(-3)

In [12]:
df['volume'] = df['volume'].apply(changeVolumeType)

In [13]:
df['volumn(M)'] = df['volume']
df = df.drop('volume', axis = 1)
df.head()

,date,price,high,low,open,change,timestamp,FOXBUSINES.text_COUNT_last1days:microsoft,FOXBUSINES.text_last1days:microsoft,transformedDoc,FOXBUSINES.text_COUNT_last1days:negative,FOXBUSINES.text_last1days:negative,WALL_STREE.text_last1days:positive,WALL_STREE.text_COUNT_last1days:positive,volumn(M)
0,2013-02-01 16:00:00,27.62,27.73,27.15,27.25,3.41,1.357142e+09,0,[],NaN,0,[],['the latest tweets and updates about the #fis...,6,52.91
1,2013-03-01 16:00:00,27.25,27.65,27.16,27.63,-1.34,1.357229e+09,2,['stocks ease after yesterdays big rally econo...,stocks ease after yesterdays big rally economy...,4,['amazons third-party sellers scored big this ...,['breaking: new jersey gov_ christie says hous...,3,48.30
2,2013-04-01 16:00:00,26.74,27.34,26.73,27.27,-1.87,1.357315e+09,2,['stocks mildly lower as traders mull mostly i...,stocks mildly lower as traders mull mostly inl...,2,['stocks mildly lower as traders mull mostly i...,['mark zuckerberg gave 498_8 million in facebo...,10,52.52
3,2013-07-01 16:00:00,26.69,26.88,26.64,26.77,-0.19,1.357574e+09,1,['stock futures edge lower after last weeks su...,stock futures edge lower after last weeks surg...,1,['stock futures edge lower after last weeks su...,"['buy a dvd or book, youve got something to pa...",5,37.12
4,2013-08-01 16:00:00,26.55,26.79,26.46,26.75,-0.52,1.357661e+09,0,[],NaN,0,[],"['@danahaggard ""assume positive intent"" seems ...",4,44.70


In [14]:
# feature engineer the text 
def textTransform(doc, max_length = 13):
    exclude = set(string.punctuation)
    doc = eval(doc)
    sw = stopwords.words('english')
    porter = PorterStemmer()
    output = ''
    if (len(doc) == 0):
        return output
    else:
        for each in doc:
            each = re.sub('[0-9]+','',each) # remove numbers
            each = ''.join(ch for ch in each if ch not in exclude) # remove puntuations and extra spaces
            output += each
        output = [x for x in word_tokenize(output) if not len(x) > max_length]
        output_without_sw = [word for word in output if not word in sw]
        stemmed = [porter.stem(word) for word in output_without_sw] 
        return stemmed

In [15]:
df['WSJ_Fox'] = df['WALL_STREE.text_last1days:positive'].apply(textTransform).apply(lambda x: " ".join(x) ) + df['FOXBUSINES.text_last1days:microsoft'].apply(textTransform).apply(lambda x: " ".join(x) )

In [16]:
# locate 2016 - 2020
df['normalizedDate'] = df.date.dt.normalize()
df['year'] = df['date'].dt.year

In [17]:
df[(df.year == 2016)].iloc[0]

date                                                                       2016-04-01 16:00:00
price                                                                                     54.8
high                                                                                      54.8
low                                                                                      53.39
open                                                                                     54.32
change                                                                                   -1.23
timestamp                                                                          1.45192e+09
FOXBUSINES.text_COUNT_last1days:microsoft                                                    2
FOXBUSINES.text_last1days:microsoft          ['china stocks slump 7 on weak data surveys ',...
transformedDoc                               china stocks slump 7 on weak data surveys us s...
FOXBUSINES.text_COUNT_last1days:negative          

In [18]:
df = df[727:]

In [19]:
temp = df[['normalizedDate', 'year', 'price', 'high', 'low', 'open', 'change',  'volumn(M)', 'WSJ_Fox']]
temp

,normalizedDate,year,price,high,low,open,change,volumn(M),WSJ_Fox
727,2016-04-01,2016,54.80,54.80,53.39,54.32,-1.23,53.78,protest occupi oregon wildlif refug ask militi...
728,2016-05-01,2016,55.05,55.39,54.54,54.93,0.46,34.08,global stock start sharp selloff today top tow...
729,2016-06-01,2016,54.05,54.40,53.64,54.32,-1.82,39.52,gun stock surg obama crack puerto rican govern...
730,2016-07-01,2016,52.17,53.48,52.07,52.70,-3.48,56.56,bangladesh suprem court uphold death sentenc i...
731,2016-08-01,2016,52.33,53.28,52.15,52.37,0.31,48.75,despit stock market turmoil trader say investo...
...,...,...,...,...,...,...,...,...,...
1883,2020-10-23,2020,216.23,216.23,213.22,215.03,0.62,18.88,slovakia began mass coronaviru test campaign t...
1884,2020-10-26,2020,210.08,216.30,208.16,213.73,-2.84,37.11,presid trump could still win piec togeth requi...
1885,2020-10-27,2020,213.25,214.67,210.33,211.65,1.51,36.70,hed better without demeanor buchanan counti va...
1886,2020-10-28,2020,202.68,208.72,202.13,207.65,-4.96,51.20,visa deal buy plaid key player space face resi...


In [20]:
# Add more features
# Dividends
msft = yf.Ticker("MSFT")

In [21]:
hist = msft.history(period="max")

In [22]:
dividends = msft.dividends.to_frame().reset_index()
dividends[dividends.Date.dt.year == 2016]

,Date,Dividends
48,2016-02-16,0.36
49,2016-05-17,0.36
50,2016-08-16,0.36
51,2016-11-15,0.39


In [23]:
dividends = dividends[49:]

In [24]:
temp = temp.merge(dividends, \
         left_on = 'normalizedDate', \
         right_on = 'Date', \
         how = 'left').drop('Date', axis = 1)

In [25]:
temp.Dividends = temp.Dividends.fillna(method = 'bfill').fillna(method = 'ffill')

In [26]:
# RSI
def rsi(price, n):
    delta = price.diff()
    dUp = delta[delta>0]
    dDown = delta[delta<0]
    dUp, dDown = delta.copy(), delta.copy()
    
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(n).mean()
    RolDown = dDown.rolling(n).mean().abs()

    RS = RolUp / RolDown
    return 100.0 - (100.0 / (1.0 + RS))

In [27]:
temp['RSI'] = rsi(df.price, 14)

In [28]:
temp['RSI'] = temp['RSI'].fillna(method = 'bfill')

In [29]:
# PE Ratio, EPS
url = 'https://www.macrotrends.net/stocks/charts/MSFT/microsoft/pe-ratio'
from bs4 import BeautifulSoup
import requests
import json

In [30]:
page = requests.get(url)
page.status_code

200

In [31]:
soup = BeautifulSoup(page.content, 'html.parser')

In [32]:
table = soup.find('table')
cols = [table.find_all('th')[x].get_text() for x in [1,2,3,4]] 
cols

['Date', 'Stock Price', 'TTM Net EPS', 'PE Ratio']

In [33]:
table_data = [[cell.text for cell in table("td")]
                         for row in table("tr")]

In [34]:
counter = 0
rows = []
newRow = []
for i in table_data[0]:
    if ((counter % 4 == 0) and (len(newRow) != 0)):
        rows.append(newRow)      
        newRow = []
    newRow.append(i)
    counter+=1

In [35]:
pe_table = pd.DataFrame(columns = cols, data = rows)
pe_table.head()

,Date,Stock Price,TTM Net EPS,PE Ratio
0,2020-12-01,216.21,,34.93
1,2020-09-30,209.77,$6.19,33.89
2,2020-06-30,202.48,$5.75,35.21
3,2020-03-31,156.48,$6.00,26.08
4,2019-12-31,156.05,$5.74,27.19


In [36]:
pe_table.dtypes

Date           object
Stock Price    object
TTM Net EPS    object
PE Ratio       object
dtype: object

In [37]:
pe_table['Date'] = pd.to_datetime(pe_table.Date)
pe_table['Stock Price'] = [eval(x) for x in pe_table['Stock Price']]
pe_table['TTM Net EPS'] = [eval(x[1:]) if x is not '' else x for x in pe_table['TTM Net EPS'] ]
pe_table['PE Ratio'] = [eval(x) for x in pe_table['PE Ratio']]

In [38]:
pe_table = pe_table.sort_values(by = 'Date', ascending = True).reset_index(drop = True)
pe_table = pe_table[37:].reset_index(drop=True)

In [39]:
pe_table

,Date,Stock Price,TTM Net EPS,PE Ratio
0,2016-06-30,47.52,2.05,23.18
1,2016-09-30,53.83,2.08,25.88
2,2016-12-31,58.46,2.12,27.58
3,2017-03-31,62.33,2.26,27.58
4,2017-06-30,65.61,2.7,24.30
5,2017-09-30,71.28,2.94,24.24
6,2017-12-31,82.26,1.46,56.34
7,2018-03-31,88.18,1.8,48.99
8,2018-06-30,95.68,2.11,45.35
9,2018-09-30,111.41,2.41,46.23


In [40]:
temp = temp.merge(pe_table, \
         left_on = 'normalizedDate', \
         right_on = 'Date', \
        how = 'left').fillna(method = 'bfill').drop(['Date', 'Stock Price'], axis = 1)

In [41]:
temp[['TTM Net EPS', 'PE Ratio']] = temp[['TTM Net EPS', 'PE Ratio']].fillna(method = 'ffill')

### Exponential Moving Average

In [42]:
# short_rolling = temp['price'].rolling(window=20).mean()
ema_short = temp['price'].ewm(span=20, adjust=False).mean()

In [43]:
temp['ema'] = ema_short

## Count vectorizer

In [44]:
countvectorizer = CountVectorizer(analyzer= 'word', \
                                  stop_words='english', \
                                  ngram_range=(1,1),\
                                 min_df = 0.03, \
                                 max_df = 0.99) 
#ignore terms in less than 5% of the documents
#ignore terms in more than 98% of the documents

In [45]:
count_wm = countvectorizer.fit_transform(temp['WSJ_Fox'])
count_tokens = countvectorizer.get_feature_names()

In [46]:
prep_df = pd.DataFrame(count_wm.toarray(), columns=countvectorizer.get_feature_names())
prep_df

,accord,account,accus,act,ad,add,advis,age,agenc,agre,...,worst,worth,write,wsj,wsjopinion,wsjwhatsnow,year,yield,york,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1157,1,0,0,0,0,0,0,1,0,0,...,0,0,1,1,0,0,1,0,0,0
1158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1159,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [47]:
prep_df[['price', 'high', 'low', 'open', 'change', 'volumn(M)',\
         'Dividends', 'RSI', 'TTM Net EPS','PE Ratio']] = temp[['price', 'high', 'low', 'open', 'change', 
                                                                'volumn(M)','Dividends', 'RSI', 'TTM Net EPS',\
                                                                'PE Ratio']]

In [48]:
# Where ‘change’ decreased more than 0.5%
prep_df['negative_poc'] = [1 if log_diff < -0.005 else 0 for log_diff in prep_df['change']]
# Where ‘change’ increased more than 0.5%
prep_df['positive_poc'] = [1 if log_diff > 0.005 else 0 for log_diff in prep_df['change']]

In [49]:
prep_df.shape

(1161, 604)

In [50]:
## autoregression that could be improved

In [51]:
# We need to shift our target one day so that we predict 
# for what will happen 'tomorrow' with news and data from 'today'
prep_df['target'] = prep_df['positive_poc'].shift(-1)
prep_df[['change', 'positive_poc', 'target']]

,change,positive_poc,target
0,-1.23,0,1.0
1,0.46,1,0.0
2,-1.82,0,0.0
3,-3.48,0,1.0
4,0.31,1,0.0
...,...,...,...
1156,0.62,1,0.0
1157,-2.84,0,1.0
1158,1.51,1,0.0
1159,-4.96,0,0.0


In [52]:
prep_df = prep_df[:-1] # Remove the last one with NaN
# Define target
target = 'target'
# Create train/test sets
X = prep_df.loc[:, prep_df.columns != target].values
y = prep_df.loc[:,[target]].values
div = int(round(len(X) * 0.71))
X_train = X[:div]
y_train = y[:div]
X_test = X[div:]
y_test = y[div:]
print('Train:')
print(X_train.shape)
print(y_train.shape)
print('Test:')
print(X_test.shape)
print(y_test.shape)

Train:
(824, 603)
(824, 1)
Test:
(336, 603)
(336, 1)


In [54]:
cor = prep_df[['price', 'high', 'low', 'open', 'change', 'volumn(M)',\
         'Dividends', 'RSI', 'TTM Net EPS','PE Ratio', 'target']]
cor[cor.columns[1:]].corr()['target'][:]

high           0.030773
low            0.031659
open           0.032263
change        -0.097692
volumn(M)     -0.046261
Dividends      0.039670
RSI            0.041874
TTM Net EPS    0.050363
PE Ratio       0.033209
target         1.000000
Name: target, dtype: float64

### Random forest regressor

In [55]:
%%time
rf = RandomForestRegressor(n_estimators = 1000, random_state = 1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


CPU times: user 26.5 s, sys: 60.7 ms, total: 26.5 s
Wall time: 26.7 s


In [56]:
print("AUC score:")
print(roc_auc_score(y_test, y_pred))
print('---')
# Let's binarize and look at the confusion matrix
preds = [1 if val > 0.5 else 0 for val in y_pred]
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test, preds))
print('---')
# Lets look at the accuracy score
print('Accuracy:')
print(accuracy_score(y_test, preds))
print('---')

AUC score:
0.48254846643518523
---
Confusion Matrix:
[[ 74  70]
 [106  86]]
---
Accuracy:
0.47619047619047616
---


### Stochastic gradient descent

In [57]:
clf = SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [58]:
print("AUC score:")
print(roc_auc_score(y_test, y_pred))
print('---')
# Let's binarize and look at the confusion matrix
preds = [1 if val > 0.5 else 0 for val in y_pred]
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test, preds))
print('---')
# Lets look at the accuracy score
print('Accuracy:')
print(accuracy_score(y_test, preds))
print('---')

AUC score:
0.5
---
Confusion Matrix:
[[144   0]
 [192   0]]
---
Accuracy:
0.42857142857142855
---


## TF-IDF vectorizer

In [59]:
tfidfvectorizer = TfidfVectorizer(analyzer='word', \
                                  stop_words= 'english', \
                                  ngram_range=(1, 1))
tfidf_wm = tfidfvectorizer.fit_transform(temp['WSJ_Fox'])
tfidf_tokens = tfidfvectorizer.get_feature_names()

In [60]:
prep_df2 = pd.DataFrame(tfidf_wm.toarray(), columns=tfidfvectorizer.get_feature_names())
prep_df2.head()

,aapl,abil,abl,abus,access,accord,account,accus,acquir,act,...,write,wsj,wsjheard,wsjopinion,wsjwhatsnow,year,yearold,yield,york,young
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.183707,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [61]:
prep_df2[['price', 'high', 'low', 'open', 'change', 'volumn(M)',\
         'Dividends', 'RSI', 'TTM Net EPS','PE Ratio']] = temp[['price', 'high', 'low', 'open', 'change', 
                                                                'volumn(M)','Dividends', 'RSI', 'TTM Net EPS',\
                                                                'PE Ratio']]

In [62]:
# Where ‘change’ decreased more than 0.5%
prep_df2['negative_poc'] = [1 if log_diff < -0.005 else 0 for log_diff in temp['change']]
# Where ‘change’ increased more than 0.5%
prep_df2['positive_poc'] = [1 if log_diff > 0.005 else 0 for log_diff in temp['change']]

In [63]:
# We need to shift our target one day so that we predict 
# for what will happen 'tomorrow' with news and data from 'today'
prep_df2['target'] = prep_df2['positive_poc'].shift(-1)
prep_df2[['change', 'positive_poc', 'target']]

,change,positive_poc,target
0,-1.23,0,1.0
1,0.46,1,0.0
2,-1.82,0,0.0
3,-3.48,0,1.0
4,0.31,1,0.0
...,...,...,...
1156,0.62,1,0.0
1157,-2.84,0,1.0
1158,1.51,1,0.0
1159,-4.96,0,0.0


In [64]:
prep_df2 = prep_df2[:-1] # Remove the last one with NaN
# Define target
target = 'target'
# Create train/test sets
X2 = prep_df2.loc[:, prep_df2.columns != target].values
y2 = prep_df2.loc[:,[target]].values
div = int(round(len(X) * 0.71))
X_train2 = X2[:div]
y_train2 = y2[:div]
X_test2 = X2[div:]
y_test2 = y2[div:]
print('Train:')
print(X_train2.shape)
print(y_train2.shape)
print('Test:')
print(X_test2.shape)
print(y_test2.shape)

Train:
(824, 900)
(824, 1)
Test:
(336, 900)
(336, 1)


### RandomForestRegressor

In [65]:
%%time
rf2 = RandomForestRegressor(n_estimators = 1000, random_state = 1)
rf2.fit(X_train2, y_train2)
y_pred2 = rf2.predict(X_test2)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


CPU times: user 57.2 s, sys: 282 ms, total: 57.5 s
Wall time: 58.5 s


In [66]:
print("AUC score:")
print(roc_auc_score(y_test2, y_pred2))
print('---')
# Let's binarize and look at the confusion matrix
preds2 = [1 if val > 0.5 else 0 for val in y_pred2]
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test2, preds2))
print('---')
# Lets look at the accuracy score
print('Accuracy:')
print(accuracy_score(y_test2, preds2))
print('---')

AUC score:
0.5278682002314815
---
Confusion Matrix:
[[ 48  96]
 [ 59 133]]
---
Accuracy:
0.5386904761904762
---


### SGDClassifier

In [67]:
clf = SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X_train2, y_train2)
y_pred2 = clf.predict(X_test2)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [68]:
print("AUC score:")
print(roc_auc_score(y_test2, y_pred2))
print('---')
# Let's binarize and look at the confusion matrix
preds2 = [1 if val > 0.5 else 0 for val in y_pred2]
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test2, preds2))
print('---')
# Lets look at the accuracy score
print('Accuracy:')
print(accuracy_score(y_test2, preds2))
print('---')

AUC score:
0.5
---
Confusion Matrix:
[[144   0]
 [192   0]]
---
Accuracy:
0.42857142857142855
---


Tf-idf and count vectorizer have almost the same accuracy. We will improve the model by adding in more text. Further, we will consider using time series to improve the accuracy.